# Medicare Fraud Detection

### BigData Query for cms-medicare data

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='bigquery-public-data')

In [ ]:
medicare_ref = client.dataset("cms_medicare")
medicare = client.get_dataset(medicare_ref)

In [ ]:
list(map(lambda table: table.table_id, client.list_tables(medicare)))

In [ ]:
table_ref = medicare_ref.table('physicians_and_other_supplier_2015')
table = client.get_table(table_ref)

In [ ]:
table.schema

In [ ]:
df = client.list_rows(table, max_results=100000).to_dataframe()

In [ ]:
df.head(10)

In [ ]:
df.groupby(['nppes_entity_code', 'medicare_participation_indicator']).count()

In [ ]:
df.loc[df.medicare_participation_indicator=='N']

In [ ]:
df.groupby(['place_of_service']).count()

In [ ]:
df.groupby(['nppes_entity_code']).count()

In [ ]:
df.groupby(['npi', 'provider_type']).count()

### Handling LEIE data

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
leie_data = pd.read_csv('../input/dataset/UPDATED.csv')

In [ ]:
leie_data.head(10)

In [ ]:
fraud_data = leie_data.loc[leie_data.NPI!=0].loc[leie_data.EXCLDATE>20150630]

In [ ]:
fraud_data.head(10)

In [ ]:
fraud_data.count()

### Required feature table preparation

In [ ]:
individual_provider_data = df.loc[df.nppes_entity_code=='I'].loc[df.medicare_participation_indicator=='Y']
individual_provider_data.count()

In [ ]:
cols = ['npi', 'bene_unique_cnt', 'bene_day_srvc_cnt', 'line_srvc_cnt', 'average_medicare_allowed_amt', 'average_submitted_chrg_amt', 'average_medicare_payment_amt', 'average_medicare_standard_amt']
all_data = individual_provider_data[cols]

In [ ]:
all_data.head(10)

Convert numeric data to numeric type

In [ ]:
print(all_data.dtypes)
all_data = all_data.apply(pd.to_numeric, errors="ignore")
dataset = all_data.loc[all_data.npi!=0]

In [ ]:
dataset.head(10)

Checking number of tuples having null values

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.count()

Aggregating expenses by npi of the provider

In [ ]:
dataset = dataset.groupby(['npi'], as_index=False).agg('sum')

In [ ]:
dataset.head(10)

In [ ]:
dataset.count()

Introducing 'fraud' label

In [ ]:
dataset['fraud'] = 0

Labelling fraud records

In [ ]:
fraud_npi = set(fraud_data.NPI)
dataset.loc[dataset.npi.isin(fraud_npi), 'fraud'] = 1

In [ ]:
dataset.columns

In [ ]:
dataset.head(10)

In [ ]:
dataset.count()

In [ ]:
dataset.loc[dataset.fraud==1].head(10)

In [ ]:
dataset.loc[dataset.fraud==1].count()

Removing npi column

In [ ]:
dataset = dataset.iloc[:, 1:]

In [ ]:
dataset.columns

In [ ]:
dataset.head(10)

### Splitting input features and corresponding labels

In [ ]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[:5]

In [ ]:
y_train = y

In [ ]:
X_train = X

In [ ]:
fraud = np.count_nonzero(y_train)
legit = y_train.shape[0]-fraud

In [ ]:
legit

In [ ]:
fraud

# Experiment Section

In [ ]:
from imblearn.pipeline import Pipeline, make_pipeline

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline = make_pipeline(StandardScaler(), SMOTE(), RandomForestClassifier())

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
evaluation = cross_validate(pipeline, X_train, y_train, scoring=['f1','accuracy','roc_auc'], cv=3)

In [ ]:
evaluation

---------------------

### Handling unbalanced class

#### No data resampling

### Scaling numeric input data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X_trs = scaler.fit_transform(X_train)

In [ ]:
X_trs[:5]

### Building and training model

#### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
classifier = GaussianNB()

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
evaluation = cross_validate(classifier, X_trs, y_train, cv=10, scoring=['roc_auc', 'f1_weighted', 'precision_weighted', 'recall_weighted'])
for metric in evaluation:
    print(metric, evaluation[metric].mean())

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
evaluation = cross_validate(classifier, X_trs, y_train, cv=5, scoring=['roc_auc', 'f1_weighted', 'precision_weighted', 'recall_weighted', 'f1', 'precision', 'recall'], return_estimator=True)
for metric in evaluation:
    if metric!='estimator':
        print(metric, evaluation[metric].mean())

In [ ]:
feature_importances = evaluation['estimator'][-1].feature_importances_

In [ ]:
plt.barh(dataset.iloc[:,:-1].columns, feature_importances)

-----------------------------------------------------------------------------------------------------

#### Random Under Sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
resampler = RandomUnderSampler(sampling_strategy=0.5)

In [ ]:
X_tr, y_tr = resampler.fit_resample(X_train, y_train)

In [ ]:
X_tr.shape

In [ ]:
fraud_tr = np.count_nonzero(y_tr)
legit_tr = y_tr.shape[0]-fraud_tr

In [ ]:
legit_tr

In [ ]:
fraud_tr

### Scaling numeric input data

In [ ]:
X_trs = scaler.fit_transform(X_tr)

In [ ]:
X_trs[:5]

### Building and training model

#### Naive Bayes

In [ ]:
classifier = GaussianNB()

In [ ]:
evaluation = cross_validate(classifier, X_trs, y_tr, cv=5, scoring=['roc_auc', 'f1_weighted', 'precision_weighted', 'recall_weighted', 'f1', 'precision', 'recall'])
for metric in evaluation:
    print(metric, evaluation[metric].mean())

#### Random Forest

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
evaluation = cross_validate(classifier, X_trs, y_tr, cv=5, scoring=['roc_auc', 'f1_weighted', 'precision_weighted', 'recall_weighted', 'f1', 'precision', 'recall'], return_estimator=True)
for metric in evaluation:
    if metric!='estimator':
        print(metric, evaluation[metric].mean())

In [ ]:
feature_importances = evaluation['estimator'][-1].feature_importances_

In [ ]:
plt.barh(dataset.iloc[:,:-1].columns, feature_importances)

-----------------------------------------------------------------------------------------------------

#### Random Over Sampling

##### ADASYN

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
from imblearn.over_sampling import ADASYN
resampler = ADASYN(sampling_strategy=0.5)

In [ ]:
X_tr, y_tr = resampler.fit_resample(Xtrain, ytrain)

In [ ]:
X_tr.shape

In [ ]:
fraud_tr = np.count_nonzero(y_tr)
legit_tr = y_tr.shape[0]-fraud_tr

In [ ]:
legit_tr

In [ ]:
fraud_tr

### Scaling numeric input data

In [ ]:
X_trs = scaler.fit_transform(X_tr)
Xtest = scaler.transform(Xtest)

In [ ]:
X_trs[:5]

### Building and training model

#### Naive Bayes

In [ ]:
classifier = GaussianNB()

In [ ]:
evaluation = cross_validate(classifier, X_trs, y_tr, cv=5, scoring=['roc_auc', 'f1_weighted', 'precision_weighted', 'recall_weighted', 'f1', 'precision', 'recall'])
for metric in evaluation:
    print(metric, evaluation[metric].mean())

#### Random Forest

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
evaluation = cross_validate(classifier, X_trs, y_tr, cv=3, scoring=['roc_auc', 'f1_weighted', 'precision_weighted', 'recall_weighted', 'f1', 'precision', 'recall'], return_estimator=True)
for metric in evaluation:
    if metric!='estimator':
        print(metric, evaluation[metric].mean())

In [ ]:
model = evaluation['estimator'][-1]

In [ ]:
feature_importances = model.feature_importances_

In [ ]:
plt.barh(dataset.iloc[:,:-1].columns, feature_importances)

In [ ]:
y_score = model.predict_proba(Xtest)[:,1]
fpr, tpr, thresholds = roc_curve(ytest, y_score)
print('AUC: {}'.format(auc(fpr, tpr)))

In [ ]:
plt.figure(figsize=(10, 8))
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='-')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
barWidth = 0.25

# set height of bar
f1 = [0.4104784852074201, 0.9952031306692811, 0.7455215205717257]
roc = [0.677015520697708, 0.9999310415789591, 0.9400921217771855]
 
# Set position of bar on X axis
br1 = np.arange(len(f1))
br2 = [x + barWidth for x in br1]
 
# Make the plot
plt.bar(br1, f1, color ='#1f77b4', width = barWidth,
        edgecolor ='grey', label ='f1-score')
plt.bar(br2, roc, color ='#7f7f7f', width = barWidth,
        edgecolor ='grey', label ='roc-auc-score')
 
# Adding Xticks
plt.xlabel('Random Forest')
plt.ylabel('Score (0-1)')
plt.xticks([r + barWidth/2 for r in range(len(f1))],
        ['RUS', 'SMOTE', 'ADASYN'])
 
plt.legend()
plt.show()

In [ ]:
barWidth = 0.25

# set height of bar
f1 = [0.1711651095702235, 0.1960470111938177, 0.18019022668742624]
roc = [0.5442282055186182, 0.5989925951759163, 0.5819544779304673]
 
# Set position of bar on X axis
br1 = np.arange(len(f1))
br2 = [x + barWidth for x in br1]
 
# Make the plot
plt.bar(br1, f1, color ='#1f77b4', width = barWidth,
        edgecolor ='grey', label ='f1-score')
plt.bar(br2, roc, color ='#7f7f7f', width = barWidth,
        edgecolor ='grey', label ='roc-auc-score')
 
# Adding Xticks
plt.xlabel('Naive Bayes')
plt.ylabel('Score (0-1)')
plt.xticks([r + barWidth/2 for r in range(len(f1))],
        ['RUS', 'SMOTE', 'ADASYN'])
 
plt.legend()
plt.show()